# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
%matplotlib inline
import torch
import torchvision
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import helper
import pandas as pd
import time
import re
import copy

from torch import nn, optim
from torchvision import datasets, transforms, models
from pathlib import Path
from PIL import Image

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [2]:
data_dir = '/home/hameed/data/flower_data'

if Path.exists(Path(data_dir)):
    data_transforms = {
        'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(p=.3),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),

        'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }

    image_datasets = {x: datasets.ImageFolder(Path(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'valid']}
    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
        
    print(f"Loaded data from {dataloaders['train'].dataset.root} and {dataloaders['valid'].dataset.root} ")
else:
    print(f"{data_dir} is a faulty path")

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [3]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Grab images and labels from trainloader
images, labels = next(iter(dataloaders['train'])) 

# labels returns the value - which is just index of the key class in the dictionary
# Use the value index to get key value then search cat_to_name
helper.imshow(images[0], title=labels[0], normalize=True)

# Grab classmap from trainload
# Find the key, based on value - which is just key index 
# The key from the class map is used in cat_to_name to get the correct corresponding name  for each flower
class_map = dataloaders['train'].dataset.class_to_idx 
index = [* class_map][labels[0].item()]
cat_to_name[index]

# Visualize Data


# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [11]:
# TODO: Save the checkpoint 
def save_model(model, optimizer,needed_epochs, model_params = None, exp_lr_scheduler=None,  file_name="checkpoint.pth"):
    
    parent_path = Path(Path.cwd(), model.name + "_checkpoints")
    if Path.exists(parent_path) == False:
        Path.mkdir(parent_path)
        
#     save_path = Path.mkdir(Path(parent_path, file_name))
    
    checkpoint = {'fc' : model.fc,
                  'optimizer' : optimizer.state_dict(),
                  'model_params' : model_params,
                  'exp_lr_scheduler' : exp_lr_scheduler.state_dict(),
                  'epochs' : needed_epochs,
                  'state_dict': model.state_dict()}
    
    torch.save(checkpoint, Path(parent_path, file_name))

# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_model(checkpoint_path, Display=False):
    """ Loads the define model with saved weights.
        
        model: Model object
        best_model_wts: The model's state_dict()
        
        Returns model with loaded weights.
    """
    if Path.exists(Path(checkpoint_path)) == False:
        print("Faulty path to checkpoint")
        return None
    
    map_location = ("cuda" if torch.cuda.is_available() else "cpu")
    checkpoint = torch.load(checkpoint_path, map_location)

    model =  models.resnet152(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    model.fc = checkpoint['fc']
    
    if Display:
        print(f"Optimizer: {checkpoint['optimizer']}")
        print(f"Epochs needed: {checkpoint['epochs']}")
    
    model.load_state_dict(checkpoint['state_dict'])
    return model
    


def log_params(data=None, file_path='paramLog.csv', columns=['accuracy',  'testing_loss', 'base_model','added_arc', 'epochs', 'batch_size', 'optimizer', 'init_lr' ]):
    """
    Use to save hyper parameters in Excel for future reference
    
    data: Hyper parameter Infomation for future reference and comparison
    file_path: desired path to save Excel file
        By default saves to 'paramLog.xlsx' in current directory
        
    columns: reference labels for records in the Excel file  
        By default ['accuracy', 'testing_loss',  'base_model','added_arc', 'epochs' ,  'batch_size', 'optimizer', 'init_lr' ]
    """
    if data.shape != (1, len(columns)):
        if data is None:
            data = np.arange(len(columns))
        data = np.reshape(data, (1, len(columns)))
    
    # Create dataframe if spreadsheet doesn't exist 
    if Path.exists(Path(file_path)) == False:
        param_log = pd.DataFrame(data, columns=columns)
        print(f"{param_log}")
        pd.DataFrame.to_csv(param_log, file_path, columns=columns, index=False) 
    else: 
        # Otherwise load the existing data
        param_log = pd.read_csv(file_path)
        pd.DataFrame.to_csv(param_log, file_path, header=False, index=False, mode='a') 
    
    if Path.exists(Path(file_path)): 
        print(f"Hyper Parameters have been written to {file_path}")

def train_model(model, criterion, optimizer, scheduler, num_epochs=30):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    model.name = re.split("\(\n", str(model), 1)[0]
    optimizer_name = re.split(' ', str(optimizer), maxsplit=1)[0]
    init_lr = optimizer.param_groups[0]['lr']
    batch_size = dataloaders['train'].batch_size
    added_layers = model.fc
    
    train_log = ""
    training_epoch_time = 0
    
    for epoch in range(num_epochs):
        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            epoch_since = time.time()
            
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            epoch_time = epoch_since - time.time()
            
            if phase == 'train':
                linebrk = '='
                training_epoch_time = epoch_time
                print(f"{linebrk * 125}", f"\nEpoch: {epoch + 1}/{num_epochs} \t{phase} Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}")
            else:
                epoch_time += training_epoch_time
                print(f"\t\t{phase} Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f} \t {epoch_time // 60:.0f}m {epoch_time % 60:.0f}")
            
            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                needed_epochs = epoch
                
                best_model_wts = copy.deepcopy(model.state_dict())
                print(f"Validation accuracy increased ({best_acc:.6f} --> {epoch_acc:.6f}).  Saving model ...")
                model_params = np.array([best_acc, epoch_loss, model.name, 
                                        added_layers, needed_epochs, batch_size ,optimizer_name , init_lr], dtype=object)
                

    time_elapsed = time.time() - since
    
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    log_params(model_params)
    # load best model weights
    model.load_state_dict(best_model_wts)
    save_model(model, optimizer, needed_epochs, model_params, exp_lr_scheduler)
    return model

In [ ]:
# Let's try ResNet-152 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
res152 = torchvision.models.resnet152(pretrained=True)
res152.name = "ResNet"

train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    print(f"Moving {res152.name} model to gpu {device}")
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    res152.cuda(device)
else:
    print("Your model is on the cpu")

# TODO: Build and train your network
# Freeze learned parameters on each layer
for params in res152.parameters():
    params.requires_grad = False

# Change the last layer according to our needs
from collections import OrderedDict
res152.fc = nn.Sequential(OrderedDict([
        ('fc1', nn.Linear(2048, 1000, bias=True)),
        ('fc1_relu', nn.ReLU()),
        ('dropout1', nn.Dropout(p=.20)),
        ('fc2', nn.Linear(1000, 102, bias=True))
     ]))


epochs = 30 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(res152.fc.parameters(), lr=0.01)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# res152 = train_model(res152, criterion, optimizer, exp_lr_scheduler, epochs)

In [ ]:
save_model(res152,optimizer,21,exp_lr_scheduler=exp_lr_scheduler)

In [13]:
# model2 = torch.nn
model2 = load_model("ResNet_checkpoints/checkpoint.pth", Display=True)


Optimizer: {'state': {}, 'param_groups': [{'lr': 0.01, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'initial_lr': 0.01, 'params': [140671254012768, 140669697819128, 140669697819560, 140669697818768]}]}
Epochs needed: 21


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model
    
        image: PIL image to be prepped for pytorch model
        Returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    # Resize the image
    new_size = 256
    width, height = image.size
    aspect_ratio = width // height
    width = new_size * aspect_ratio
    height = new_size

    image = image.resize((width, height))
    
    # Crop Center 
    new_width = new_height = 224
    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2

    image = image.crop((left, top, right, bottom))
    
    # Convert pil image to numpy array and reduce channel infomation
    np_image = np.array(image) / 255

    # Normalize the image
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    np_image = (np_image - mean) / std
    
    # Reorder the color channel as first dimension
    np_image = np.transpose(np_image, (2, 0, 1))
    
    image = torch.from_numpy(np_image)
    
    return image


def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = ((std * image)) + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    try:
        class PathError(Exception):
            pass

        if Path.exists(Path(f"{image_path}")) == False:
            raise PathError()
        
        # Preprocess image
        img = Image.open(image_path)
        img = process_image(img)
        img = img.view( -1 , *img.shape)
        print(f"Image shape : {img.shape}")
        
        # Use model for inference
        # set to eval mode
        model.eval()
        with torch.no_grad():
            # Send image to device
            # Forward pass through model
            # Get a ranked prob/classes
            img = img.to(dtype= torch.float, device=device)
            output = model(img)
            prob, classes = torch.topk(output, topk, dim=1)
        return prob, classes
        
        
    except PathError as err:
        print(f"{image_path} is faulty. Try a different path {err}")
        raise
        
    
    # TODO: Implement the code to predict the class from an image file

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
image = Image.open("test_im.jpeg")
image = process_image(image)
imshow(image)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
prob, classes = predict("test_im.jpeg", res152)
prob, classes

In [ ]:
# TODO: Display an image along with the top 5 classes
prob, classes = prob.to('cpu'), classes.to('cpu')

cls_name = np.array(classes, dtype=str)
cls_name = [cat_to_name[x.item()] for x in np.nditer(cls_name)]

figure, ax = plt.subplots(2,1)
plt.subplot(2, 1, 1)
imshow(image, plt)
plt.subplot(2, 1, 2)
plt.barh(cls_name, prob.numpy()[0])
plt.show()